# 📊 대학별 연도별 통합 데이터 전처리 템플릿
2010~2023년 대학별 공시 데이터를 기준으로 연도별 + 학교명 유사도 기반 병합, 피처 통합, 결측치 처리 등

In [ ]:
import pandas as pd
from difflib import get_close_matches
from functools import reduce

## 🔹 기준 파일: 중도탈락율 불러오기 및 기준 학교명 정의

In [ ]:
# 기준 데이터 로딩
base_df = pd.read_csv("중도탈락율_통합_2010_2023.csv", encoding="utf-8-sig")
base_df.rename(columns={base_df.columns[0]: "기준년도", base_df.columns[1]: "학교명"}, inplace=True)
base_df = base_df[(base_df["기준년도"] >= 2010) & (base_df["기준년도"] <= 2023)].copy()
base_df["학교명_정규"] = base_df["학교명"]

## 🔹 연도별 유사도 기반 학교명 매핑 함수 정의

In [ ]:
standard_schools_by_year = base_df.groupby("기준년도")["학교명"].unique().to_dict()
def match_school(row, 기준년도, 기준학교명리스트):
    name = row["학교명"]
    if pd.isna(name) or 기준년도 not in 기준학교명리스트:
        return None
    candidates = 기준학교명리스트[기준년도]
    match = get_close_matches(name, candidates, n=1, cutoff=0.85)
    return match[0] if match else None

## 🔹 피처 파일 불러오기 및 병합

In [ ]:
feature_files = {
    "기숙사수용률": "기숙사_수용률_2010_2023_통합_UTF8.csv",
    "전임교원_1인당학생수": "전임교원_1인당학생수_2010_2023_통합.csv",
    "진학률_졸업자수": "졸업자수_진학률_2010_2023_유사통합_누락제거_utf8sig.csv",
    "신입생충원률_경쟁률": "통합_신입생_충원률_및_경쟁률_2010_2023_정제본_CP949.csv",
    "재학생충원율": "통합_재학생_충원율_2010_2023_CP949.csv",
    "강좌수": "학생규모별_강좌수_통합_2010_2023.csv"
}
merged_df = base_df[["기준년도", "학교명", "학교명_정규", "중도탈락율"]].copy()
for name, path in feature_files.items():
    df = pd.read_csv(path, encoding="utf-8-sig" if 'utf8' in path.lower() else "cp949")
    df.rename(columns={df.columns[0]: "기준년도", df.columns[1]: "학교명"}, inplace=True)
    df = df[(df["기준년도"] >= 2010) & (df["기준년도"] <= 2023)]
    df["학교명_정규"] = df.apply(lambda row: match_school(row, row["기준년도"], standard_schools_by_year), axis=1)
    df.dropna(subset=["학교명_정규"], inplace=True)
    df.drop(columns=["학교명"], inplace=True)
    merged_df = pd.merge(merged_df, df, how="outer", on=["기준년도", "학교명_정규"])

## 🔹 병합 후 학교명 복원 및 정리

In [ ]:
school_name_map = base_df[["기준년도", "학교명_정규", "학교명"]].drop_duplicates()
merged_df = pd.merge(merged_df, school_name_map, how="left", on=["기준년도", "학교명_정규"])
merged_df.drop(columns=["학교명_x", "학교명_정규"], errors='ignore', inplace=True)
merged_df.rename(columns={"학교명_y": "학교명"}, inplace=True)
merged_df.sort_values(by=["기준년도", "학교명"], inplace=True)

모든 연도에 존재하는 학교만 유지

In [ ]:
# import pandas as pd

# # 데이터 불러오기
# df = pd.read_csv("해당 파일.csv")

# # 2010~2023 모든 연도에 존재하는 학교명 찾기
# school_year_matrix = df.pivot_table(index="학교명", columns="기준년도", values="중도탈락율", aggfunc="count")
# all_years = list(range(2010, 2024))
# schools_all_years = school_year_matrix.dropna(subset=all_years)

# # 해당 학교명만 필터링
# filtered_df = df[df["학교명"].isin(schools_all_years.index)]

# # 기준년도 + 학교명 기준으로 중복 제거
# deduped_df = filtered_df.drop_duplicates(subset=["기준년도", "학교명"], keep="first")

# # 결과 저장
# deduped_df.to_csv("중복제거_모든연도학교204개_UTF8BOM.csv", index=False, encoding="utf-8-sig")
# print("저장 완료: 중복제거_모든연도학교204개_UTF8BOM.csv")

유사도 기반 병합 후 204개 학교만 유지 및 중복 제거 수행 업데이트

In [ ]:
# import pandas as pd
# from difflib import get_close_matches

# # 데이터 불러오기
# base_df = pd.read_csv("중복제거_모든연도학교204개_UTF8BOM.csv")
# scholarship_df = pd.read_csv("2010~2023학교별 장학금 액.csv").rename(columns={"기준연도": "기준년도"})
# employment_df = pd.read_csv("2010~2023학교별 졸업생 취업률.csv")

# # 유사도 기반 병합 함수
# def match_school_names(target_names, base_names):
#     mapping = {}
#     for name in target_names:
#         match = get_close_matches(name, base_names, n=1, cutoff=0.8)
#         if match:
#             mapping[name] = match[0]
#     return mapping

# base_school_names = base_df["학교명"].unique()
# scholarship_df["학교명_매핑"] = scholarship_df["학교명"].map(match_school_names(scholarship_df["학교명"].unique(), base_school_names))
# employment_df["학교명_매핑"] = employment_df["학교명"].map(match_school_names(employment_df["학교명"].unique(), base_school_names))

# # 병합
# merged = base_df.merge(scholarship_df[["기준년도", "학교명_매핑", "1인당장학금"]],
#                        left_on=["기준년도", "학교명"], right_on=["기준년도", "학교명_매핑"], how="left").drop(columns="학교명_매핑")
# merged = merged.merge(employment_df[["기준년도", "학교명_매핑", "취업률"]],
#                       left_on=["기준년도", "학교명"], right_on=["기준년도", "학교명_매핑"], how="left").drop(columns="학교명_매핑")

# # 중복 제거
# merged = merged.drop_duplicates(subset=["기준년도", "학교명"], keep="first")

# # 컬럼 순서 조정 및 float 변환
# cols = [col for col in merged.columns if col != "학교명"] + ["학교명"]
# merged = merged[cols]
# for col in merged.columns:
#     if col != "학교명":
#         merged[col] = pd.to_numeric(merged[col], errors='coerce')

# # 저장
# merged.to_csv("최종_통합데이터_중복제거완료_204학교기준.csv", index=False, encoding="utf-8-sig")
# print("최종 데이터 저장 완료")